In [16]:
import numpy as np
import scipy.optimize as opt
from scipy.stats import poisson
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [30]:
class Baseline:
  def __init__(self, C: np.ndarray, V: int, G:int, w: int):
    '''
    input:
    C = distribution of crime. 
        C is 2darray where the row represents C_i.
        Each index of C_i represents the number of crime
        and the value represents frequency.
    V = the total number of units of resources.
    G = the number of regions.

    c = #. crime types
    output:
    W_opt = optimal allocation of V
    X_max = the maximized objective value
    '''
    self.C = C  # (c, G)
    self.max_c = 30
    self.V = V
    self.G = G
    self.w = w #(c,)
    self.V_opt = np.zeros((self.G))
    self.X_max = 0
    self.f = np.ones((self.G, self.V+1))
    self.T = np.zeros((self.G, self.V+1))

    self.calc_f()
    self.calc_T()

  def disc(self, v_i: int, c_i: list):
    '''
    Calculate discovery function.
    Input: 
    v_i = the number of patrol allocated to region i (G_i).
    c_i = a list(the number occurences of each crime in region i.)
    
    Output:
    number of discovered crime.
    '''
    disc_val = 0
    sum_c = 0
    for i, w in enumerate(self.w):
      sum_c += w*c_i[i]
    if sum_c > 0:
      disc_val = min(v_i, sum_c)/sum_c
    return disc_val
  
  def calc_f(self):
    '''
    Calculate f_i(v_i). where f_i(v_i) = E[min(v_i, sum(w_j*c_ij))/sum(w_j*c_ij)]
    Output:
    self.f = probability of discovering crime for allocating v_i patrols in region i.
    '''
    # C = num of crimes * num of regions
    # f = num of regions * (V+1)
    for g in range(self.G):
      lamb1 = self.C[0][g]
      lamb2 = self.C[1][g]
      for v in range(self.V+1):
        self.f[g, v] = 0
        for c1 in range(self.max_c):
          for c2 in range(self.max_c):     
            prob1 = poisson.pmf(k=c1, mu=lamb1)
            prob2 = poisson.pmf(k=c2, mu=lamb2)
            self.f[g, v] += self.disc(v, [c1, c2])*prob1*prob2
    return

  def calc_T(self):
    for g in range(self.G):
      prob = 0
      lamb1 = self.C[0][g]
      lamb2 = self.C[1][g]
      self.T[g][0] = 1
      for c in range(1, self.max_c+1):
        for c_next in range(c-1):
          prob += (poisson.pmf(k=c_next, mu=lamb1)*poisson.pmf(k=c-1, mu=lamb2))
          prob += (poisson.pmf(k=c_next, mu=lamb2)*poisson.pmf(k=c-1, mu=lamb1))
        prob += (poisson.pmf(k=c-1, mu=lamb1)*poisson.pmf(k=c-1, mu=lamb2))
        self.T[g][c] = 1-prob

  def get_j_star(self, v:np.ndarray, ub:np.ndarray):
    '''
    Get j_star.
    Input:
    v = the number of patrol for all region.
    ub = upper bounds for all region.
    '''
    max_val, j_star = -10e9, 0
    for j in range(self.G):
      if v[j] < ub[j]:
        diff_tau = self.T[j][v[j]+1] - self.T[j][v[j]]
        if max_val <= diff_tau:
          max_val = diff_tau
          j_star = j
    return j_star
  
  def found_bound(self, region, alpha, f_i):
    #self.f shape: (self.G, self.V+1)
    lb = self.V
    ub = 0
    for v in range(self.V):
      if self.f[region, v] >= f_i - alpha and self.f[region, v] <= f_i:
        if v < lb:
          lb = v
        elif v > ub:
          ub = v
    return lb, ub


  def algorithm_1(self, alpha: float):
    '''
    Run algorithm1 that computes an optimal fair allocation
    in the precision model
    Input:
    alpha = fairness threshold [0,1]
    V = the number of patrol for all region.
    '''
    self.X_max = 0
    for i in range(self.G):
      v = np.zeros(self.G, dtype=int)
      ub = np.zeros(self.G)
      lb = np.zeros(self.G)
      for v_i in range(self.V+1):
        #print('regin: %d, v: %d'%(i, v_i))
        v[i] = v_i
        # compute f_i(v_i).
        ub[i] = v[i] # upper bound on allocation to group i
        lb[i] = v[i] # lower bound on allocation to group i
        for j in range(self.G):
          if i == j:
            continue
          # update lower and upper bounds on allocation to group j
          lb[j], ub[j] = self.found_bound(j, alpha, self.f[i, v_i])
          v[j] = lb[j]
        # check whether v exceed #. available patrollers
        if sum(v) > self.V:
          break
        #allocating the remaining patrollers
        V_r = int(self.V - sum(v))
        for t in range(1, V_r+1):
          j_star = self.get_j_star(v, ub)
          v[j_star] += 1
        # calculating the obj value
        curr_obj = 0
        for g in range(self.G):
          for vg in range(v[g]):
            curr_obj += self.T[g][vg]
        
        #print('current allocation: ', v)
        #print('current obj value: ', curr_obj)
        if curr_obj > self.X_max:
          #print('Update maximum of X...')
          self.X_max = curr_obj
          for g in range(self.G):
            self.V_opt[g] = v[g]
    return 

In [18]:
# C for BATTERY
battery = [3.7402269861286253, 5.075662042875158, 6.796973518284994, 7.498108448928121, 
    5.572509457755359, 8.282471626733923, 7.366960907944515, 6.674653215636822, 
    5.215636822194199, 6.181588902900378, 8.184110970996217, 4.411097099621689, 
    2.5422446406052965, 5.750315258511979, 3.3619167717528375, 2.6557377049180326, 
    3.399747793190416, 3.3354350567465323, 2.0756620428751575, 3.308953341740227, 
    3.490542244640605, 5.851197982345523, 0.005044136191677175]
theft = [8.098360655737705, 5.128625472887768, 4.28499369482976, 4.542244640605296, 
    3.382093316519546, 5.5498108448928125, 3.5384615384615383, 6.055485498108449, 
    4.0226986128625475, 3.459016393442623, 3.948297604035309, 8.43001261034048, 
    5.6683480453972255, 2.8234552332912988, 4.544766708701135, 3.9029003783102145, 
    9.194199243379572, 8.871374527112232, 3.1740226986128626, 3.14249684741488, 
    4.9167717528373265, 5.326607818411097, 0.0037831021437578815]

C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [2, 1]
alpha = 0.5

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

In [19]:
print(baseline.V_opt, baseline.X_max)

[3. 3. 3. 3. 2. 4. 3. 4. 2. 3. 4. 4. 2. 2. 2. 1. 4. 4. 1. 1. 2. 3. 0.] 59.97941676729582


In [31]:
# C for BATTERY
battery = [3.7402269861286253, 5.075662042875158, 6.796973518284994, 7.498108448928121, 
    5.572509457755359, 8.282471626733923, 7.366960907944515, 6.674653215636822, 
    5.215636822194199, 6.181588902900378, 8.184110970996217, 4.411097099621689, 
    2.5422446406052965, 5.750315258511979, 3.3619167717528375, 2.6557377049180326, 
    3.399747793190416, 3.3354350567465323, 2.0756620428751575, 3.308953341740227, 
    3.490542244640605, 5.851197982345523, 0.005044136191677175]
theft = [8.098360655737705, 5.128625472887768, 4.28499369482976, 4.542244640605296, 
    3.382093316519546, 5.5498108448928125, 3.5384615384615383, 6.055485498108449, 
    4.0226986128625475, 3.459016393442623, 3.948297604035309, 8.43001261034048, 
    5.6683480453972255, 2.8234552332912988, 4.544766708701135, 3.9029003783102145, 
    9.194199243379572, 8.871374527112232, 3.1740226986128626, 3.14249684741488, 
    4.9167717528373265, 5.326607818411097, 0.0037831021437578815]

C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [2, 1]
alpha = 0
baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)
print(baseline.V_opt, baseline.X_max)

In [24]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [2, 1]
alpha = 1
baseline.algorithm_1(alpha)
print(baseline.V_opt, baseline.X_max)

[3. 3. 3. 3. 2. 4. 3. 4. 2. 3. 4. 4. 2. 2. 2. 1. 4. 4. 1. 1. 2. 3. 0.] 59.97941676729582


In [29]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [2, 1]
alpha = 0
baseline.algorithm_1(alpha)
print(baseline.V_opt, baseline.X_max)

[3. 3. 3. 3. 2. 4. 3. 4. 2. 3. 4. 4. 2. 2. 2. 1. 4. 4. 1. 1. 2. 3. 0.] 59.97941676729582


In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [5, 1]
alpha = 1

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

alpha = 0.5
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

alpha = 0
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [5, 1]
alpha = 1

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [5, 1]
alpha = 0

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [1, 1]
alpha = 1

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [1, 1]
alpha = 0.5

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
C = np.array([battery, theft])
V = 60 # number of patrol resources
G = 23 # 24 districts
w = [1, 1]
alpha = 0

baseline = Baseline(C, V, G, w)
baseline.algorithm_1(alpha)

print(baseline.V_opt, baseline.X_max)

In [ ]:
battery = [3.7402269861286253, 5.075662042875158, 6.796973518284994, 7.498108448928121, 
    5.572509457755359, 8.282471626733923, 7.366960907944515, 6.674653215636822, 
    5.215636822194199, 6.181588902900378, 8.184110970996217, 4.411097099621689, 
    2.5422446406052965, 5.750315258511979, 3.3619167717528375, 2.6557377049180326, 
    3.399747793190416, 3.3354350567465323, 2.0756620428751575, 3.308953341740227, 
    3.490542244640605, 5.851197982345523, 0.005044136191677175]
theft = [8.098360655737705, 5.128625472887768, 4.28499369482976, 4.542244640605296, 
    3.382093316519546, 5.5498108448928125, 3.5384615384615383, 6.055485498108449, 
    4.0226986128625475, 3.459016393442623, 3.948297604035309, 8.43001261034048, 
    5.6683480453972255, 2.8234552332912988, 4.544766708701135, 3.9029003783102145, 
    9.194199243379572, 8.871374527112232, 3.1740226986128626, 3.14249684741488, 
    4.9167717528373265, 5.326607818411097, 0.0037831021437578815]

sum_val = []
index = len(battery)
for i in range(index):
    sum_val.append(sum(battery[i]+theft[i]))